Groundwater | Case Study

# Topic 1 : Introduction to the Groundwater Course - The Limmat Valley Aquifer 

Dr. Xiang-Zhao Kong, Dr. Beatrice Marti and Louise Noël du Payrat

In [ ]:
# Setting up the notebook
import sys
import os
import pandas as pd
import numpy as np

from IPython.display import display
import rasterio
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

# Load local modules
sys.path.append('../SUPPORT_REPO/src')
sys.path.append('../SUPPORT_REPO/src/scripts/scripts_exercises')
import climate_utils as cu
import print_images as du
import river_utils as ru
from data_utils import download_named_file
from map_utils import (
    display_groundwater_resources_map, 
    plot_model_area_map, 
    display_concessions_map, 
    create_interactive_dem_map, 
)
from progress_tracking import (
    create_perceptual_model_progress_tracker,
    create_section_completion_marker,
)

## Introduction

Most exercises, as well as the final project in this course, are based on case studies. Case studies are real-world scenarios that provide you with an opportunity to apply your knowledge and skills to solve complex groundwater problems. In this year, we focus on the Limmat Valley Aquifer in Zurich, Switzerland. Reserve around 2 hours to read though this notebook.

We will follow the actual steps a professional groundwater modeller might take to set up a numerical model to answer a specific question. In the first step, the modellers familiarize themselves with the Limmat Valley Aquifer and its hydrological and hydrogeological properties. We start out with a very generic perceptual model of the aquifer (see Figure 1), which we'll refine as we uncover new information.


In [ ]:
du.display_image(
    image_filename='perceptual_model_00_initial.png', 
    image_folder='1_perceptual_model', 
    caption='Figure 1: Initial perceptual model of the Limmat valley aquifer (indicated in brown) which will be refined and used as a basis for a first long-term average water balance estimation. The purple arrows indicate important groundwater fluxes. Qin represent lateral inflows, while Qout represents outflows. Qpump represents groundwater extraction. Qrech and Qleak stand for areal recharge and leakage from surface water bodies respectively. ∆S stands for the storage change in the aquifer.'
)

We first aim to understand the hydrogeology of the Limmat Valley Aquifer. Next, we examine each water balance component to develop an initial estimate of the key fluxes within the system. Finally, we will revisit and refine the perceptual model as we prepare to construct a numerical model of the Limmat Valley Aquifer.

## 1 - The Limmat Valley Aquifer

The Limmat Valley Aquifer is a well-studied groundwater reservoir beneath the city of Zurich, Switzerland. According to Doppler and colleagues, it was formed during the last ice age as the Linth glacier retreated. The aquifer has no direct hydraulic connection to Lake Zurich in the east, where it is bounded by impermeable lake sediments and moraine material. It is further confined in the north and south by the side moraines of the Linth glacier. Lateral inflow of groundwater from the surrounding hills is expected. The aquifer is also hydraulically connected to the river Sihl in the east and the river Limmat in the north. Its hydraulic properties are highly heterogeneous due to a complex geological history shaped by various sedimentation and erosion events from the rivers Sihl and Limmat [\[1, 2\]](#References).

In Figure 2, you see a screenshot of the [GIS-browser](https://geo.zh.ch/maps?x=2677655&y=1253620&scale=396217&basemap=arelkbackgroundzh) of the canton of Zurich [\[3\]](#References). The cyan-blue area in the center of the map shows the Limmat Valley Aquifer. The darker the color, the greater the thickness of the aquifer. The GIS-browser is only available in German, but you can use your browser's translation feature to view it in your preferred language.

In [ ]:
du.display_image(
    image_filename='GIS-browser_canton_Zurich.png', 
    image_folder='1_perceptual_model',
    caption='Figure 2: Printscreen of the GIS-browser of the canton of Zurich displaying the cantonal groundwater map. Source: https://www.gis.zh.ch/, accessed 2024-05-01'
)

> **Action point:**  
> Take a few minutes of your time to browse the available data layers in the GIS-browser. There are many! Focus on the layers that are relevant to the Limmat valley aquifer.

We now begin systematically reviewing the available information to refine our perceptual model of the Limmat Valley Aquifer. The GIS-browser will help us clarify the aquifer's geometry and inform how we represent it in the numerical model.

You can use the checklist below to track your progress.

In [ ]:
create_perceptual_model_progress_tracker()

## 2 - Aquifer Geometry

Most layers in the GIS-Browser of the Canton of Zurich are publicly available for download (see the "Datenbezug" button in the upper right corner of the map pane). We use this feature to import the relevant data into our JupyterHub environment.

### 2.1 - Bottom Topography

We have downloaded the aquifer thickness layer from the GIS-browser and clipped it to the area of interest (AOI) for the Limmat Valley Aquifer. Let's examine the bottom topography of the aquifer in Figure 3.


In [ ]:
# This function downloads the groundwater map of the canton of Zurich and saves 
# it to the folder applied_groundwater_modelling_data in your home directory. 
gw_map_path = download_named_file(
    name='groundwater_map_norm', 
    data_type='gis', 
)


In [ ]:
# Create and display the interactive map
interactive_map = display_groundwater_resources_map(
    gw_map_path, zoom_level=11, 
    map_title="Figure 3: Groundwater resources map of the canton of Zurich",)
interactive_map

We can use contours of the aquifer thickness to build the bottom of the aquifer model (this is done in a future chapter : Notebook 4, model implementation).  

### 2.2 - Top Topography
The Limmat Valley Aquifer is unconfined. We can use the topography of the Limmat Valley to represent the top of the aquifer. swisstopo provides digital elevation models (DEMs) of Switzerland in various resolutions. We use the 25m resolution DEM for our case study. We downloaded the DEM from the [website of the Federal Office of Topography swisstopo](https://www.swisstopo.admin.ch/en/geodata/height/dhm25.html)[\[4\]](#References) and clipped it to the area of interest (AOI) of the Limmat Valley Aquifer (Figure 4). If you are interested in learning more about the DEM and how to clip it to the AOI, you can read the
[processing_DEM.ipynb](../SUPPORT_REPO/src/scripts/scripts_limmat_data_preprocessing/processing_DEM.ipynb) notebook.

In [ ]:
dem_path = download_named_file(
    name='dem',
    data_type='gis'
)

In [ ]:
m = create_interactive_dem_map(
    dem_path=dem_path, 
    zoom_start=12,
    container_title="DEM Controls",
    custom_title="Figure 4: Digital elevation model in the area of interest. Background: OpenStreetMap"
)
m

The topography of the Limmat valley is quite flat, with a maximum elevation of about 400 m above sea level. As we use a rather coarse DEM resolution of 25 m, we cannot see the small topographic features in the Limmat valley, such as the river beds. As we would like to model the river-aquifer interaction, we need to find a suitable compromise between DEM (and model grid) resolution and computational capacities to represent the river beds in our model.

River-aquifer interaction is predominantly driven by the stream bed elevation relative to the groundwater table. Therefore, an accurate representation of the streambed elevation in the numerical groundwater flow model is crucial. With a spatial resolution of 20 m, the DEM is not sufficient to represent the river beds in the Limmat valley aquifer. We will discuss the [river-aquifer interaction](#4---River-aquifer-interaction) in a later chapter, where we will also discuss how to represent the river beds in our model for different modeling purposes in the case studies.



### 2.3 - Lateral Model Boundaries
The groundwater map of the Limmat valley aquifer shows that the aquifer is not hydraulically connected to Lake Zurich in the east, but that it is bounded by impermeable lake sediments and moraine material. We will neglect aquifer parts east of the river Sihl as it is not the focus of the model goal, and we expect recharge from the river to dominate over flow from the east. The aquifer is further confined in the north and south by the side moraines of the Linth glacier. Lateral inflow of groundwater from the hills in the north and south is to be expected. The most common approach is to use a water balance approach on the hillsides to estimate the subsurface inflow from the hills. We'll address this in the chapter on [climate data](#3---Climate-Forcing). We do not consider the lateral inflow from the Sihl valley aquifer into the Limmat valley aquifer, as the inflow is rather small and parallel to the general flow direction in the Limmat valley.

Should you have to estimate the lateral inflow from the Sihl valley aquifer, you can apply Darcy's law under the Dupuit assumption. In practice, the organization tasking you to produce a groundwater flow model will typically provide you with estimates of hydraulic conductivities from field measurements. As a first estimate, you can use the hydraulic conductivities from the literature (e.g. Freeze and Cherry, 1979, shown in Figure 5)[\[5\]](#References).

In [ ]:
du.display_image(
    image_filename='hydraulic_conductivities_by_Freeze.png', 
    image_folder='1_perceptual_model',
    caption='Figure 5: Hydraulic conductivity ranges as presented in the book <<Groundwater>> by Freeze and Cherry (1979). The ranges are based on the hydraulic conductivity of unconsolidated sediments. Hydraulic conductivity is typically known only in approximate orders of magnitude.'
)


> 📚 **Theory reminder**: Applying Darcy's Law in Unconfined Aquifers: The Dupuit Approximation
>
> To apply Darcy's Law for calculating the total discharge through an unconfined aquifer, we often rely on a set of simplifying assumptions known as the *Dupuit approximation* (or Dupuit-Forchheimer assumption). These assumptions allow us to treat the complex three-dimensional flow problem as a simpler two-dimensional one by integrating flow over the entire aquifer thickness.
> 
> The key conditions that must be met for the Dupuit approximation to be valid are:
> 
> *   *The water table gradient is small:* The slope of the phreatic surface must be gente. This is the most critical assumption, as it implies that flow is almost entirely horizontal.
> *   *Flow is horizontal:* It is assumed that the hydraulic head is constant with depth for any given vertical line. This means equipotential lines are vertical, and therefore, the flow lines are horizontal.
> *   *The hydraulic gradient is equal to the slope of the water table:* The slope of the water table is assumed to be the hydraulic gradient (`i`) for throughout the saturated thickness of the aquifer.
> 
> When these conditions hold, we can use a modified form of Darcy's Law to calculate the discharge `Q` of the aquifer:
> 
> `Q = -K * h * B * (dh/dx)`
> 
> Where:
> *   `Q` is the discharge (e.g., m³/s)
> *   `K` is the hydraulic conductivity (e.g., m/s)
> *   `h` is the saturated thickness (m)
> *   `B` is the width of the aquifer perpendicular to the flow direction (m)
> *   `dh/dx` is the slope of the water table, which represents the hydraulic gradient.
> 
> In essence, the Dupuit approximation simplifies reality by ignoring vertical flow components, which is reasonable when the water table is nearly flat relatively to the saturated thickness.

For the outflow, a fixed-head boundary is typically chosen. This requires a known groundwater table at the boundary, e.g. a monitoring piezometer. Another option is to use the water level of a receiving stream or lake as a fixed-head boundary condition. The outflow boundary should be placed far enough away from the area of interest, so that the groundwater flow in the area of interest is not influenced by the boundary condition. For a regional model, this is typically several hundred meters to several kilometers away from the area of interest. 

In our case, we choose the groundwater level at piezometer 481 (indicated with a pink arrow in Figure 6) as the fixed-head boundary condition. This relieves us from the need to implement the more complex aquifer geometry and river-aquifer interaction further downstream in the Limmat valley aquifer. We can use the isoline of equal groundwater level at 390 m a.s.l. from the GIS browser to define the outflow boundary.

In [ ]:
du.display_image(
    image_filename='Detail_outflow.png', 
    image_folder='1_perceptual_model',
    caption='Figure 6: Detail of the outflow boundary condition at piezometer 481 (purple arrow). The groundwater level at this piezometer is used as a fixed-head boundary condition for the groundwater flow model. The outflow boundary is placed far enough away from the area of interest, so that the groundwater flow in the area of interest is not influenced by the boundary condition.'
)

Note that the isolines of equal groundwater levels are not perfectly perpendicular to the flow direction but slightly curved at the aquifer boundaries (particularly visible at the northern boundary). This indicates lateral groundwater inflow from the north.  

We can now make a rough estimate of the groundwater flux at the western outflow boundary of the Limmat Valley Aquifer using the Dupuit's assumption. We use the groundwater levels isolines of 389 m a.s.l. and 391 m a.s.l. to estimate the hydraulic gradient. The distance between the two isolines can be measured in a GIS tool and is about 780 m. The thickness of the aquifer in that location is between 2 m and 10 m, and the width is approximately 1 km. We estimate the hydraulic conductivity to be around 10^-2 m/s so we can estimate the hydraulic gradient as follows:


In [ ]:
outflow_lower_bound = (391 - 389) / 780 * 2 * 1000 * 0.01 * 3600 * 24 # m3/day
outflow_upper_bound = (391 - 389) / 780 * 10 * 1000 * 0.01 * 3600 * 24 # m3/day

print(f"Estimated groundwater outflow at the western boundary of the Limmat valley aquifer: {outflow_lower_bound:.2f} m3/day to {outflow_upper_bound:.2f} m3/day")

We now have an understanding of the geometry of the aquifer. Let's simplify update our virtual perceptual model with the information we have gathered so far: 

- The aquifer does not have a direct hydraulic connection to lake Zurich in the east. 
- It is probably heavily influenced by the river Sihl in the east and the river Limmat in the north.
- Since groundwater flow in river valleys generally follows the topography, we can assume that the groundwater flow in the Limmat valley aquifer is directed from south-east to west. This general flow direction is corroborated by the groundwater flow arrows which become visible when we zoom in on the GIS-browser map (visit the GIS browser to zoom in).
- The aquifer thickness is highly variable. To adequately represent the known geometry of the aquifer, a 3D model with a high spatial resolution is required. We will first opt for a 2D model with a simplified geometry for the case study work to keep the model lean and fast on JupyterHub.
- The simplified geometry will show a larger thickness in the south-east with a gradual thinning towards the west. The thickness will be represented by a single layer with a variable thickness. We will use the thickness values from the GIS-browser to define the thickness of the aquifer in our model whereby we will have to map the isolines of groundwater thickness to the model grid.
- The aquifer extends beyond the boundaries of the city of Zurich. Since we are interested mainly in applying our numerical experiment in the city region, this means that we will have to make assumptions about the outflow boundary of the aquifer.

 

### 2.4 - Drawing the model boundary polygon
We can now draw the model boundary polygon manually in QGIS and export it as a geopackage layer. Given the complex aquifer thickness, it is the fastest method. This layer will define the model area in our numerical model.

If you need instructions of how to use QGIS to draw and edit a polygon, please refer to resources available online. 

In [ ]:
model_boundary_path = download_named_file(
    name='model_boundary',
    data_type='gis'
)

# Get the area of the model domain
gdf_boundary = gpd.read_file(model_boundary_path)
model_area_m2 = gdf_boundary.geometry.area.sum()
model_area_km2 = model_area_m2 / 1e6  # Convert to km²
print(f"Model area: {model_area_km2:.2f} km²")  

plot_model_area_map(
    gw_depth_path=gw_map_path, 
    model_boundary_path=model_boundary_path,
    custom_title="Figure 18: Model region with the model boundary in black. Base layer by OpenStreetMap.",
    basemap="osm", 
    basemap_alpha=0.9
)

In [ ]:
create_section_completion_marker(1)  # Mark section 1 complete

## 3 - Climate Forcing
Climate data is used to : 
- estimate the aquifer recharge from precipitation infiltrating. 
- estimate the lateral inflow from the hills in the north and south of the valley. 

### 3.1 - Areal Recharge
We get climate data from the Swiss Federal Office of Meteorology and Climatology (MeteoSwiss). The data is available for viewing on [MeteoSwiss](https://www.meteoswiss.admin.ch/services-and-publications/applications/measurement-values-and-measuring-networks.html#param=messnetz-klima&table=false&station=SMA&compare=y&chart=year) website [\[6\]](#References) and was downloaded via the [opendata.swiss plattform](https://opendata.swiss/en/dataset/klimanormwerte) [\[7\]](#References). It is available in the data directory of the case study repository. 
The closest station to the Limmat Valley Aquifer is Fluntern. We use data from there, which is available for the time period 1991-2020.

In [ ]:
# Get climate data 
climate_data_path = download_named_file(
    name='climate_data', 
    data_type='climate', 
)
# If it is a zip file, unzip it
if climate_data_path.endswith('.zip'):
    import zipfile
    with zipfile.ZipFile(climate_data_path, 'r') as zip_ref:
        extract_path = os.path.dirname(climate_data_path)
        zip_ref.extractall(extract_path)
    climate_data_path = extract_path  # Update path to the extracted folder
# Read and process climate data
climate_norms = cu.read_climate_data(climate_data_path)
# Inspect the climate norms
display(climate_norms)

> **🤔 Think about it**  
> We are looking at the climate data mostly to get an idea about groundwater recharge. In an aquifer located in a densely populated area, we have to consider that the recharge is not only influenced by the climate but also by human activities. What do you think are the most important human activities that influence groundwater recharge in this area?

Figure 8 below shows the monthly average temperature and precipitation for the Fluntern station. 

In [ ]:
plt, fig = cu.plot_climate_data(
    climate_norms, 
    custom_title="Figure 8: Climate data for the Fluntern station (source ).")
plt.show()

Now we need to get from climate data to net recharge at the groundwater table, taking into account the mostly built-up land cover. You have several options to do this which are listed below by increasing complexity :  

- Use net recharge literature values in the project area, or reported fractions from precipitation in comparable areas ( ie. of similar climatic and land cover conditions)
- Estimate net recharge using a water balance approach (precipitation minus evaporation)
- Use a land surface model (e.g. Hydrus)
- Simulate the water flow through the unsaturated zone in Modflow (requires high-resolution 3D grid for numerical stability)

We start out with the less complex option. Lehner suggests that between 5% and 15% of the precipitation infiltrates into the aquifer [\[8\]](@references). We will use a value of 10% as a first approximation, ie. 110 mm/year.
The rest of the precipitation evaporates (approximately 20% according to Qui et al. (2023)) or runs off into the Sihl or the Limmat (via the rainwater drainage system). 

From net recharge, once the area of the model domain is known, can be further calculated the areal net recharge to the aquifer. 

### 3.2 - Lateral Inflow from Hills
The lateral inflow from the hills is estimated using a water balance approach on the hill sides. 

Information needed are :
- climate data (as for the areal recharge in the previous paragraph)
- hills area, which is estimated using QGIS (see Figure 10)

We assume that 10% of the precipitation in a hill area infiltrates and is transported, eventually becoming lateral inflow into the aquifer.

In [ ]:
du.display_image(
    image_filename='rough_first_manual_catchment_delineation_to_estimate_area_for_lateral_inflow.png', 
    image_folder='1_perceptual_model',
    caption='Figure 10: Manual catchment delineation to estimate area for lateral inflow. The area is delineated using the height model of the area (here DHM200). The area is used to estimate the lateral inflow from the hills in the north and south of the Limmat valley aquifer. The delineation is not precise and should be refined in a later step. The area relevant for the lateral inflow estimation is 15 km2 in the south and 11 km2 in the north of the Limmat valley aquifer.'
)

Please note, this manual delineation is sufficient for a first rough estimation. A proper catchment delineation would require a more detailed analysis of the topography and the hydrological conditions in the area. You find step-by-step instructions on how to delineate a catchment in many places on the internet, for example here: [https://hydrosolutions.github.io/caham_book/geospatial_data.html#sec-catchment-delineation](https://hydrosolutions.github.io/caham_book/geospatial_data.html#sec-catchment-delineation). 



In [ ]:
inflow_south = 0.1 * 15e6  # m3/year
inflow_north = 0.1 * 11e6  # m3/year

print(f"Estimated lateral inflow from the south: {(inflow_south/1000000):.1f} 10^6 m³/year")
print(f"Estimated lateral inflow from the north: {(inflow_north/1000000):.1f} 10^6 m³/year")

> **Think about it:**. 
> What type of boundary condition would you use to represent the lateral inflow from the hills in a steady state numerical model? Why?  


In [ ]:
create_section_completion_marker(2)  # Mark section 2 complete

## 4 - River-aquifer interaction

In most unconsolidated rock aquifers in river valleys, river-aquifer interaction is a key process that influences groundwater levels and flow patterns. The interaction between the river and the aquifer can lead to changes in water levels in the aquifer, which can affect the availability of groundwater resources. We therefore look at this process in more detail: 

1. Understand the river geometry and its interaction with the aquifer (chapters [4.1](#41---Introduction-to-River-Geometry--Recharge) and [4.2](#42---Interactive-Exploration-of-River-Aquifer-Interaction)).
2. Analyze monitoring data for river discharge and river water levels (chapter [4.3](#43---Understanding-Monitoring-Data-River-Discharge--River-Water-Levels)).
3. Estimate the leakage flux between the river and the aquifer (chapter [4.4](#44---Estimating-the-Leakage-Flux)).

### 4.1 - Introduction to River Geometry & Recharge
The river-aquifer interaction is an important component of the groundwater flow model. The geometry of the river and its interaction with the aquifer influence both the direction of groundwater flow and the groundwater levels in the aquifer.

In practice, a modeler uses measured profiles of the riverbed to represent the river geometry in the model. For this course, we do not have access to riverbed profiles so we must make some assumptions about the river geometry.

If you are based in Zurich, you can visit the rivers, for example on a bicycle tour. Visiting your model area is a great way to get a feel for the site and understand the hydrological processes at play. If you don't have the opportunity to visit the area, you can use the GIS-browser or download selected tiles of the high-resolution elevation model to get an idea of the river geometry and ask a local project partner to help you build the perceptual model. 


### 4.2 - Interactive Exploration of River-Aquifer Interaction

Figure 11 below provides an interactive tool to help you visualize the complex relationship between a river and an adjacent unconfined aquifer. It consists of two main parts:

1.  **Left Plot (Physical Cross-Section):** This shows a physical model of the river and the groundwater system. You can see the water level in the river (`H_riv`) and the water table in the aquifer (`H_aq`).
2.  **Right Plot (Flux Relationship):** This conceptual graph shows the calculated flux (flow rate, Q_riv) between the river and the aquifer as a function of the aquifer head. The red dot indicates the current state of the system shown on the left.

**How to Use This Figure:**

Use the two sliders below the plot to change the **Aquifer Head (`H_aq`)** and the **River Stage (`H_riv`)**. As you move the sliders, observe how the system changes in both plots. Pay close attention to the title, the equation, the direction of the flow (indicated by the arrow), and the position of the red dot on the flux curve.

**Scenarios to Explore:**

Try to create the following three fundamental conditions:

1.  **Gaining Stream:**
    *   **How to create it:** Set the aquifer head (`H_aq`) to be *higher* than the river stage (`H_riv`).
    *   **What to observe:** Water flows from the aquifer into the river. The flux (`Q_riv`) is negative (flow *out of* the groundwater). The head difference (`ΔH`) is between `H_aq` and `H_riv`. Nb : on the right plot, the red dot is in the negative flux region.

2.  **Losing Stream (Connected):**
    *   **How to create it:** Set the river stage (`H_riv`) to be *higher* than the aquifer head (`H_aq`), but keep `H_aq` *above* the riverbed bottom (`R_bot`, the dashed brown line).
    *   **What to observe:** Water flows from the river into the aquifer. The flux is positive and its magnitude depends directly on the head difference (`ΔH`). As you lower `H_aq`, the flux increases. The red dot on the right plot moves down the sloped part of the curve.

3.  **Losing Stream (Disconnected):**
    *   **How to create it:** Lower the aquifer head (`H_aq`) until it is *below* the riverbed bottom (`R_bot`).
    *   **What to observe:** A yellow "Vadose Zone" appears between the riverbed and the water table, indicating they are no longer directly connected. The flow from the river now seeps downward at a constant, maximum rate. The flux no longer depends on `H_aq`. Notice that the head difference (`ΔH`) for the calculation is now between `H_riv` and `R_bot`. On the right plot, the red dot moves onto the vertical part of the curve, showing that the flux remains constant even if you continue to lower `H_aq`.

In [ ]:
ru.plot_river_aquifer_interaction(
    custom_title="Figure 11: Illustration of river-aquifer interaction and the corresponding flux vs. head relationship."
)

### 4.3 - Understanding Monitoring Data: River Discharge & River Water Levels
The BAFU offers access to many hydrological data for Switzerland. You can find surface water monitoring sites under [map.geo.admin.ch](https://map.geo.admin.ch), under layer *Hydrological gauging stations* [\[10\]](#References) (see Figure 12). 

In [ ]:
du.display_image(
    image_filename='BAFU_HydrologicalGaugingStations.png', 
    image_folder='1_perceptual_model',
    caption='Figure 12: Hydrological gauging stations layer available at map.geo.admin.ch.' 
)

When you zoom in to the Limmat Valley Aquifer (Figure 13), you find the gauging stations of the rivers Sihl and Limmat. The Limmat gauging station located in Baden is downstream of a run-by-the-river hydropower plant, and therefore not relevant for our study. Relevant stations are the Sihl gauging station in the city of Zurich (right upstream of the confluence with the river Limmat) and the Limmat gauging station also in the city of Zurich (right downstream of the confluence with the Sihl). 

In [ ]:
du.display_image(
    image_filename='BAFU_HydrologicalGaugingStations_closeup.png',
    image_folder='1_perceptual_model',
    caption='Figure 13: Locations of hydrological gauging stations near the Limmat valley aquifer.', 
)

A click on the gauaging station brings you directly to the station site made available by the BAFU. The gauging station on the Sihl is called [*Sihl - Zürich, Sihlhölzli*](https://www.hydrodaten.admin.ch/de/seen-und-fluesse/stationen-und-daten/2176) and has ID 2176 and the gauging station on the river Limmat is called [*Limmat - Zürich, Unterhard*](https://www.hydrodaten.admin.ch/de/seen-und-fluesse/stationen-und-daten/2099) and has ID 2099. The IDs are unique numeric station identifiers, and are typically required to retrieve data from data repositories or APIs. 

> **🤔 Think about it**  
> For surface water balancing, discharge is the important variable. However, when it comes to flooding or river-aquifer interaction, the water level is the more important variable. Why do you think that is?

From the station sites, we see current water levels, typically over the past 7 days but no water level dynamics. We had to request this data from BAFU. Please note that data for recent years data is provisional and subject to change.

Let's have a look at the data (Figures 14 and 15 and Table below). 


In [ ]:
# Download river data
river_data_path = download_named_file(
    name='river_data', 
    data_type='time_series', 
)
# Unzip if necessary
if river_data_path.endswith('.zip'):
    import zipfile
    with zipfile.ZipFile(river_data_path, 'r') as zip_ref:
        extract_path = os.path.dirname(river_data_path)
        zip_ref.extractall(extract_path)
    river_data_path = extract_path  # Update path to the extracted folder

# Check if path exists
if not os.path.exists(river_data_path):
    print(f"Path {river_data_path} does not exist.")
else:
    try:
        # Plot the typical yearly evolution of river water levels
        fig, axes = ru.plot_yearly_river_levels(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020,
            figsize=(12, 8), 
            figure_number=14  # Set figure number for the plot
        )
        
        # Display the plot
        plt.show()
        
        # Get summary statistics
        summary = ru.get_river_level_summary(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020
        )
        
        print("\n=== RIVER WATER LEVEL SUMMARY (2010-2020) ===")
        print(f"\nSihl River ({summary['sihl']['station_name']}):")
        print(f"  Mean water level: {summary['sihl']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['sihl']['min']:.3f} - {summary['sihl']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['sihl']['std']:.3f} m")
        print(f"  Total range: {summary['sihl']['range']:.3f} m")
        
        print(f"\nLimmat River ({summary['limmat']['station_name']}):")
        print(f"  Mean water level: {summary['limmat']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['limmat']['min']:.3f} - {summary['limmat']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['limmat']['std']:.3f} m")
        print(f"  Total range: {summary['limmat']['range']:.3f} m")
        
    except ImportError as e:
        print(f"Plotting functionality not available: {e}")
        print("However, we can still analyze the data...")
        
        # Get summary statistics even without plotting
        summary = ru.get_river_level_summary(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020
        )
        
        print("\n=== RIVER WATER LEVEL SUMMARY (2010-2020) ===")
        print(f"\nSihl River ({summary['sihl']['station_name']}):")
        print(f"  Mean water level: {summary['sihl']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['sihl']['min']:.3f} - {summary['sihl']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['sihl']['std']:.3f} m")
        print(f"  Total range: {summary['sihl']['range']:.3f} m")
        
        print(f"\nLimmat River ({summary['limmat']['station_name']}):")
        print(f"  Mean water level: {summary['limmat']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['limmat']['min']:.3f} - {summary['limmat']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['limmat']['std']:.3f} m")
        print(f"  Total range: {summary['limmat']['range']:.3f} m")

# Save the river data summary to a CSV file
summary_df = np.save(
    os.path.join(river_data_path, 'river_data_summary.npy'),
    summary
)

Additional river gauging stations are maintained by the AWEL (cantonal office for waste, water, energy and air (Amt für Abfall, Wasser, Energie und Luft (AWEL))) [\[11\]](#References). 

### 4.4 - Estimating the Leakage Flux
We need to compare river water levels and groundwater levels, to determine if the streams are gaining or losing water relatively to the aquifer and to estimate the water flux between the rivers and the aquifer.

In [ ]:
# Sihl River Data
sihl_gw_mean = 400.8
sihl_gw_high = 403.5
sihl_river_mean = 412.347
sihl_river_high = 413.439
sihl_depth_mean = 0.3

# Limmat River Data
limmat_gw_mean = 399.5
limmat_gw_high = 400.5
limmat_river_mean = 400.285
limmat_river_high = 401.822
limmat_depth_mean = 0.7

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14))
fig.suptitle("Figure 16: River and Groundwater Level Cross-Sections", fontsize=16, y=0.95)

# Plot Sihl
ru.plot_cross_section(ax1, "a) River Sihl", sihl_gw_mean, sihl_gw_high, sihl_river_mean, sihl_river_high, sihl_depth_mean)

# Plot Limmat
ru.plot_cross_section(ax2, "b) River Limmat", limmat_gw_mean, limmat_gw_high, limmat_river_mean, limmat_river_high, limmat_depth_mean)

plt.tight_layout(rect=[0, 0, 1, 0.93]) # Adjust layout to make space for suptitle
plt.show()

From the two conceptualized cross-sections at the gauge location on the river Sihl we see that we are dealing with a losing stream which is disconnected from the groundwater table. We therefore have a constant flux from the river into the aquifer. The flux is not influenced by the groundwater table in the aquifer. The flux is influenced only by the river stage and the river bed elevation.
The river Limmat is also a loosing stream at the location of the gauge. However, from the conceptualized cross-section we see that the river Limmat is very likely connected to the groundwater table. We have to account for a capillary fringe of up to 30 cm (depending on the pore size distribution of the soil matrix). The flux from the river into the aquifer is influenced by the groundwater table in the aquifer. The flux is further influenced by the river stage and the river bed elevation.

The river-aquifer interaction is limited by the hydraulic conductivity and the thickness of the clogging layer (colmation layer), a bio-geochemically highly active sediment layer which forms on river beds over time. The clogging layer is typically a few centimeters thick and has a significantly lower hydraulic conductivity than the underlying aquifer material. It therefore limits the flux from the river into the aquifer. Doppler and colleagues [\[12\]](#References) discuss the implementation of the river-aquifer interaction in their groundwater flow model of the Limmat valley aquifer. 

$$q_{riv} = \frac{K_{Schmutzschicht}}{d_{Schmutzschicht}} \cdot (H_{riv} - H_{aq}) = l_{leakage} \cdot (H_{riv} - H_{aq})$$

Where:
- $q_{riv}$ is the flux from the river into the aquifer ($m$ $s^{-1}$)
- $K_{Schmutzschicht}$ is the hydraulic conductivity of the Schmutzschicht ($m$ $s^{-1}$)
- $d_{Schmutzschicht}$ is the thickness of the clogging layer of the river bed ($m$) 
- $l_{leakage}$ is the leakage coefficient ($s^{-1}$)
- $H_{riv}$ is the river water level ($m$)
- $H_{aq}$ is the groundwater level in the aquifer ($m$)

Doppler and colleages calibrated leakage coefficients of $1.3 \cdot 10^{-6}$ $1/s$ for the river Sihl and between $3.5 \cdot 10^{-7}$ $1/s$ and $3.5 \cdot 10^{-5}$ $1/s$ for the river Limmat.  

To simplify the first rough estimation of groundwater recharge from river infiltration, we assume :
- disconnected streams for both rivers 
- an average river water level of 0.3 m to 0.5 m , respectively for the rivers Sihl and Limmat

This yields the following specific fluxes from the rivers to the aquifer: 



In [ ]:
q_riv_sihl = 1.3e-6 * (sihl_river_mean - sihl_gw_mean)  # m/s
q_riv_limmat = 3.5e-6 * (limmat_river_mean - limmat_gw_mean)  # m/s

print(f"Estimated specific flux from Sihl to aquifer: {q_riv_sihl:.2e} m/s")
print(f"Estimated specific flux from Limmat to aquifer: {q_riv_limmat:.2e} m/s")

The flux from the Limmat is expected to be to the upper end of the range, because the river is in some stretches connected to the groundwater table. The aquifer might exfiltrate into the Limmat in downstream stretches but those are far away from our focus area.     

The area of the rivers Sihl and Limmat we can estimate from the geopackage layer "Oeffentliche Oberflaechengewaesser" (see Figure 17) provided by the Department of Geoinformation of the Canton of Zurich ([https://opendata.swiss/en/dataset/offentliche-oberflachengewasser](https://opendata.swiss/en/dataset/offentliche-oberflachengewasser), accessed 2025-07-06) using QGIS. Once we have determined the model boundary and estimated the area of the river beds within the model boundary, we can estimate a recharge volume from the rivers to the aquifer. 

In [ ]:
# Download the shapes of the rivers and the locations of the gauging stations. 
gauges_file_path = download_named_file(
    name='gauges',
    data_type='gis'
)
rivers_file_path = download_named_file(
    name='rivers',
    data_type='gis'
)
plot_model_area_map(
    gw_depth_path=gw_map_path, 
    rivers_path=rivers_file_path,
    gauges_path=gauges_file_path,
    custom_title="Figure 17: Model region with surface water bodies (labeled <<Rivers>>) and groundwater gauging stations (labeled <<Gauges>>).",
)

Now we cut the rivers to get their area within the model boundary. The river area is needed to estimate the recharge from the rivers into the aquifer.

In [ ]:
# Get the path to the river shapes and to the boundary outline
river_data_path = download_named_file(name='rivers', data_type='gis')
boundary_path = download_named_file(name='model_boundary', data_type='gis')

# Intersect the river data with the model grid and only keep the parts that are 
# inside the model boundary. 
river_gdf = gpd.read_file(river_data_path)
boundary_gdf = gpd.read_file(boundary_path)
# Clip the river data to the model boundary
river_clipped = gpd.clip(river_gdf, boundary_gdf)

# Print the column names of the clipped river data to understand its structure
# print("Clipped river data columns:")
# print(river_clipped.columns)
# Print the unique values in the 'GEWAESSERNAME' column to understand the river names
# print("\nUnique river names in the clipped data:")
# print(river_clipped['GEWAESSERNAME'].unique())

# We are interested in the river sections belonging to the rivers Sihl and Limmat. 
# We will further keep the canal in the north of the Werdinsel. This canal does 
# not have a name in the river data, but we can identify it through the geometry 
# and the unique OBJID. 
river_clipped = river_clipped[
    (river_clipped['GEWAESSERNAME'].isin(['Sihl', 'Limmat'])) |
    (river_clipped['OBJID'].isin(['32998', '34996', '37804', '95527']))
]

# Calculate the area of the clipped river sections for Sihl and Limmat
sihl_area = river_clipped[river_clipped['GEWAESSERNAME'] == 'Sihl'].geometry.area.sum()
limmat_area = river_clipped[river_clipped['GEWAESSERNAME'] == 'Limmat'].geometry.area.sum()
canal_area = river_clipped[river_clipped['OBJID'].isin(['32998', '34996', '37804', '95527'])].geometry.area.sum()

print(f"Area of Sihl within model boundary: {sihl_area:.0f} m²")
print(f"Area of Limmat within model boundary: {limmat_area:.0f} m²")
print(f"Area of canal within model boundary: {canal_area:.0f} m²")

# Plot the clipped river data to verify
fig, ax = plt.subplots(figsize=(12, 12))
river_clipped.plot(ax=ax, color='blue', linewidth=2, label='Clipped River Data')
blue_polygon = mpatches.Patch(facecolor='blue', linewidth=2, label='Clipped River Data')
boundary_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
red_line = mlines.Line2D([], [], color='red', linewidth=2, label='Model Boundary')
ax.set_title(f"Figure 19: Clipped river data within the model boundary. The areas of the river beds are: Sihl: {sihl_area:.0f} m², Limmat: {limmat_area:.0f} m², Canal: {canal_area:.0f} m²", fontsize=14)
ax.set_xlabel("X-coordinate")
ax.set_ylabel("Y-coordinate")
ax.set_aspect('equal', adjustable='box')
ax.legend(handles=[blue_polygon, red_line])
plt.show()

The canal in the north of the Werdinsel is downstream of our area of interest and lined with concrete, so we do not consider it in our model. A first estimate of the leakage inflow from the rivers Sihl and Limmat to the aquifer is now:

In [ ]:
river_sihl_inflow = sihl_area * q_riv_sihl * 365 * 24 * 3600  # m³/year
river_limmat_inflow = limmat_area * q_riv_limmat * 365 * 24 * 3600  # m³/year

print(f"Estimated groundwater recharge from Sihl: {(river_sihl_inflow/1000000):.1f} 10^6 m³/year")
print(f"Estimated groundwater recharge from Limmat: {(river_limmat_inflow/1000000):.1f} 10^6 m³/year")

In [ ]:
create_section_completion_marker(3)

## 5 - Groundwater pumping
The Limmat valley aquifer is heavily used for groundwater abstraction. The groundwater is used for drinking water supply but also for industrial purposes and for cooling. The actual pumping and infiltration rates are not publicly available. To estimate the total groundwater abstraction, we get the list of active and former concessions for groundwater abstractions and recharges from the GIS-browser of the Canton of Zurich. The layer is called "Grundwasserfassungen" and can be found in the layer list of the GIS-browser. It lists the maximum allowable rates. We'll have a look at the layer in Figure 20. 

As there are many active and inactive concessions in the Limmat valley aquifer, we will concentrate on the large concessions and screen the medium sized concessions in order to not miss any significant abstractions. For this present study, we will neglect small concessions. Here are the steps we take to estimate groundwater abstraction rates in the Limmat valley:
1. Inspect the "Grundwasserfassungen" & understand the layer in the GIS-browser for active and former concessions.
2. Extract the maximum allowable abstraction rates for the large and medium concessions.
3. Sum the maximum allowable abstraction rates for all large and medium concessions to get the total groundwater abstraction for the Limmat valley.

### 5.1 Inspect "Grundwasserfassungen"

In [ ]:
# Get the path to the well locations
well_data_path = download_named_file(name='wells', data_type='gis')

# Read the geopackage into a geopandas dataframe
wells_gdf = gpd.read_file(well_data_path, layer='GS_GRUNDWASSERFASSUNGEN_OGD_P')

# Inspect the table
wells_gdf.head()

Let's have a look at the columns:    
- N: Northing
- E: Easting
- GWR_ID: ID of the groundwater concession (wells with the same GWR_ID are part of the same concession)
- FASSBEZ: Description of the location of the well
- FASSART: Type of the well (e.g. vertical or horizontal well)
- NUTZART: Use of the well
- BESCHREIBUNG: Description of allowed pumping amounts

N, E, GWR_ID, NUTZART, and BESCHREIBUNG are relevant for us. Let's inspect the values in these columns for further analysis.

In [ ]:
# Print unique values in relevant columns
print("Unique NUTZART values:", wells_gdf['NUTZART'].unique())

NUTZART contains many cryptic abbreviations. We'll provide a mapping of these abbreviations to their full descriptions for better understanding below. The abbreviations are specific for the canton of Zurich and may be different in other regions. 

In [ ]:
# Map the unique values of NUTZART to their full descriptions
# You don't need to learn this mapping. They are just here for your reference. 
nutzart_mapping = pd.DataFrame([
    ("TW",   "Trinkwasser",                 "Drinking water"),
    ("BW",   "Brauchwasser",                "Process water / Non-potable water"),
    ("BEW",  "Bewässerung",                 "Irrigation"),
    ("GWE",  "Gewerbe",                     "Commercial use"),
    ("GWEIN","Gewerbe und Industrie",       "Commercial and industrial use"),
    ("KW",   "Kühlwasser",                  "Cooling water"),
    ("NWV",  "Natur und Wasserhaushalt / Versickerung", "Ecological / recharge use"),
    ("SA",   "Sickeranlage",                "Infiltration system"),
    ("WPG",  "Wärmepumpe Grundwasser",      "Groundwater heat pump"),
    ("WE",   "Wellness / Erholung",         "Wellness / recreational use"),
    ("WPZ",  "Wärmepumpen-Zirkulation",     "Heat pump circulation"),
], 
columns = ["Abbreviation", "German", "English"])

print(nutzart_mapping.to_string(index=False))

We are interested in larger concessions in the shallow aquifer part. We look for unique concession IDs before the "_" character in the GWR_ID column. Let's check how many unique concession IDs we have within the boundaries of our model. We will restrict our search for active wells only so we remove all rows that contain the string "aufgehoben" (decommissioned) or "ungenutzt" (not used) in the column BESCHREIBUNG. 

In [ ]:
# Clip the GeoDataFrame to the model boundaries
wells_gdf = wells_gdf.clip(boundary_gdf)

# Add a concession_id (part before first "_")
wells_gdf['concession_id'] = wells_gdf['GWR_ID'].str.split('_').str[0]

# Flag decommissioned and unused (case-insensitive match)
is_decommissioned = wells_gdf['BESCHREIBUNG'].str.contains('aufgehoben', case=False, na=False)
is_unused = wells_gdf['BESCHREIBUNG'].str.contains('ungenutzt', case=False, na=False)

# Split into active / decommissioned
wells_active = wells_gdf[~is_decommissioned & ~is_unused].copy()
wells_decommissioned = wells_gdf[is_decommissioned].copy()

# Unique concession ids
all_concessions = wells_gdf['concession_id'].nunique()
active_concessions = wells_active['concession_id'].nunique()
decomm_concessions = wells_decommissioned['concession_id'].nunique()

print(f"Total concessions (within boundary): {all_concessions}")
print(f"Active concessions: {active_concessions}")
print(f"Decommissioned concessions: {decomm_concessions}")

There are 67 active concessions within the boundaries of our model. Let's see how many concessions we have in each category in column BESCHREIBUNG.  

In [ ]:
# Count the unique concession_id of active_concessions for each unique value in BESCHREIBUNG
active_concessions_count = wells_active.groupby('BESCHREIBUNG')['concession_id'].nunique()

print(f"Number of active concessions by BESCHREIBUNG: {active_concessions_count}")

### 5.2 -  Focus on the largest concessions

In [ ]:
id_col    = "concession_id"   # adjust if different
desc_col  = "BESCHREIBUNG"
use_col   = "NUTZART"

phrase_high      = "Grundwasserfassung mit Ertrag > 3000 l/min"
phrase_recharge  = "Grundwasseranreicherungsanlage, Rückversickerung, Sickergalerie"

# 1. Keep only wells with a non-empty NUTZART (still “active” set assumed already)
wells_valid = wells_active[
    wells_active[use_col].notna() &
    wells_active[use_col].astype(str).str.strip().ne("")
].copy()

# 2. Identify high-yield wells
mask_high = wells_valid[desc_col].str.contains(phrase_high, case=False, na=False)
high_yield_wells = wells_valid[mask_high]

# 3. Concessions having at least one high-yield well
concessions_with_high = (
    high_yield_wells[id_col]
    .dropna()
    .astype(str)
    .unique()
)

# 4. Subset ALL wells belonging to those concessions (this is the expanded set)
large_concessions = wells_valid[
    wells_valid[id_col].astype(str).isin(concessions_with_high)
].copy()

# 5. Classify well role inside those concessions
large_concessions["WELL_GROUP"] = np.select(
    [
        large_concessions[desc_col].str.contains(phrase_high, case=False, na=False),
        large_concessions[desc_col].str.contains(phrase_recharge, case=False, na=False),
    ],
    ["HighYield", "Recharge"],
    default="OtherWithinConcession"
)

# (Optional) If you want to keep only wells that are either HighYield or Recharge
# plus all other wells in those *same* concessions (already done above).
# If you instead want to drop the 'OtherWithinConcession' group, uncomment:
# large_concessions = large_concessions[large_concessions['WELL_GROUP'] != 'OtherWithinConcession'].copy()

print(f"Concessions with high-yield wells: {len(concessions_with_high)}")
print("Counts by WELL_GROUP:")
print(large_concessions["WELL_GROUP"].value_counts())


We highlight these large concessions in an interactive map. The color of the concessions indicate the NUTZART (usage type) of the well. Small active concessions are not shown in this map.  

In [ ]:
# Display large_concessions and color the dots by NUTZART. 
# The map should be interactive. Hovering over the dots should display more information about the concession.
# Add the boundary_gdf to the map and use openstreetmap as the base layer.
display_concessions_map(
    large_concessions, 
    boundary_gdf=boundary_gdf, 
    map_title="Figure 20: Wells belonging to large concessions (> 3000 l/min) in the model area colored by concession ID and marked by use type. Small active concessions are not shown in this map."
)


We observe intensive thermal use of groundwater in the Limmat Valley aquifer (concession type "WPG", shown in blue, orange, green, and violet in Figure 20). Thermal use of groundwater is typically non-consumptive in the Limmat Valley aquifer: it is abstracted in one well (labeled "Entnahme" in the FASSART column) and returned to the aquifer in another well after use (labeled "Rückgabe" in the "FASSART" column). The FASSART can be viewed by hovering the cursor over the well.

For the purpose of this instructive model, we will not yet implement existing heat extraction and reinjection wells. Instead, we will focus on the main abstraction wells, which are located in the Hardhof well field (concession b1-71, olive green in Figure 20).

Hardhof is one of four pillars of the drinking water supply of the City of Zurich [\[13\]](#references) and contributes an average of 15% of the annual supply (most of the city’s drinking water comes from treated lake water). In the Hardhof well field, riverbank filtrate is extracted along the River Limmat, infiltrated into 3 recharge basins and 12 infiltration wells, and finally collected by four horizontal wells to be distributed into the drinking water network. The Hardhof site is quite complex to model, and we will not be able to fully reproduce its hydrogeology with our simplified model. Nevertheless, Hardhof is of great interest, as it employs an operational groundwater flow model that assimilates groundwater head observations to optimize infiltration in the vertical wells in real time. In fact, this is the site where real-time groundwater modeling and well field control were first developed more than a decade ago. According to the most recent statistical yearbook of the City of Zurich, the water works produce around 7 million cubic meters of groundwater per year [\[14\]](#references).  

The concession is for 104,000 L/min (GIS-Browser), but only about 13% of this concessioned volume is actually extracted. The concession therefore represents only the upper limit of possible groundwater abstraction.

> **Suggestion:**. 
> If you are located in Zurich, we highly encourage you to visit the Hardhof well field on one of their free public guided tours.

Next, let’s examine the medium-sized concessions to ensure we do not overlook important fluxes. We will neglect the small concessions, as their pumping rates are below 300 L/min, which lies well within our uncertainty range.

### 5.3 -  Focus on the medium concessions

In [ ]:
id_col    = "concession_id"   # adjust if different
desc_col  = "BESCHREIBUNG"
use_col   = "NUTZART"

phrase_medium      = "Grundwasserfassung mit Ertrag 300 - 3000 l/min"
phrase_recharge  = "Grundwasseranreicherungsanlage, Rückversickerung, Sickergalerie"

# 1. Keep only wells with a non-empty NUTZART (still “active” set assumed already)
wells_valid = wells_active[
    wells_active[use_col].notna() &
    wells_active[use_col].astype(str).str.strip().ne("")
].copy()

# 2. Identify medium-yield wells
mask_medium = wells_valid[desc_col].str.contains(phrase_medium, case=False, na=False)
medium_yield_wells = wells_valid[mask_medium]

# 3. Concessions having at least one medium-yield well
concessions_with_medium = (
    medium_yield_wells[id_col]
    .dropna()
    .astype(str)
    .unique()
)

# 4. Subset ALL wells belonging to those concessions (this is the expanded set)
medium_concessions = wells_valid[
    wells_valid[id_col].astype(str).isin(concessions_with_medium)
].copy()

# 5. Classify well role inside those concessions
medium_concessions["WELL_GROUP"] = np.select(
    [
        medium_concessions[desc_col].str.contains(phrase_medium, case=False, na=False),
        medium_concessions[desc_col].str.contains(phrase_recharge, case=False, na=False),
    ],
    ["MediumYield", "Recharge"],
    default="OtherWithinConcession"
)

# (Optional) If you want to keep only wells that are either HighYield or Recharge
# plus all other wells in those *same* concessions (already done above).
# If you instead want to drop the 'OtherWithinConcession' group, uncomment:
# medium_concessions = medium_concessions[medium_concessions['WELL_GROUP'] != 'OtherWithinConcession'].copy()

print(f"Concessions with medium-yield wells: {len(concessions_with_medium)}")
print("Counts by WELL_GROUP:")
print(medium_concessions["WELL_GROUP"].value_counts())

# Count the number of concessions for TW and BW
print("Number of concessions for TW and BW:")
print(medium_concessions[medium_concessions[use_col].isin(["TW", "BW", "TW, BW"])][id_col].nunique())

display_concessions_map(
    medium_concessions, 
    boundary_gdf=boundary_gdf, 
    map_title="Figure 21: Wells belonging to medium concessions (300 - 3000 l/min) in the model area colored by concession ID and marked by use type. Small and large active concessions are not shown in this map."
)


Most of the medium-sized concessions again involve heat extraction for district heating systems, which rely on groundwater abstraction and reinjection. Therefore, no significant consumptive use is expected. There are five concessions for drinking water and industrial water use (TW and BW). Let’s take a closer look at them.

In [ ]:
# List the active wells in medium concessions which are used for drinking water and industrial water use
active_wells_medium_concessions = medium_concessions[medium_concessions[use_col].isin(["TW", "BW", "TW, BW"])]
print(active_wells_medium_concessions[['GWR_ID', 'FASSBEZ', 'FASSART', 'NUTZART']])

By clicking on a well in the GIS-Browser, we can access slightly more information than what is provided in the geodata layer available here. The Lochergut concession (b010063) is for 520 L/min, and the Schlachthof concession is for 370 L/min. The concessioned pumping rate for concession b010113 is 2000 L/min. The two drinking water wells north of the Limmat (concessions n010039_01 and n010085_01) allow pumping of 1200 L/min and 3000 L/min, respectively.

Since the two drinking water wells north of the river are most likely pumping riverbank filtrate and are located far downstream of our focus area, we will not implement them for now. As we observed at Hardhof, only a fraction of the concessioned amount is typically extracted. For the purpose of this model, we will assume 40%. Under this assumption, the medium-sized concessions south of the Limmat would amount to about 600,000 m³/year (less than 10% of the Hardhof abstractions).

In [ ]:
outflow_pumping = 7600000  # m3/year

# Since we don't know the exact groundwater abstractions, we'll also have to 
# estimate the uncertainty of our estimate. 
# We'll assume a conservative uncertainty of 20%.

In [ ]:
create_section_completion_marker(4)  # Mark section 4 complete

## 6 - Monitoring the Limmat Valley Aquifer

To gain a first impression of the groundwater levels in the Limmat valley aquifer, we will have a look at the groundwater map.

Several authorities do groundwater monitoring in the Limmat valley aquifer. We will start with the federal office for the environment (FOEN) which maintains a network of groundwater observation wells. You find an overview over the available groundwater monitoring sites at [https://map.geo.admin.ch/](https://map.geo.admin.ch/) in layer *Groundwater level/spring discharge* [\[13\]](#References). One monitoring well maintained by FOEN is located in the Limmat valley aquifer but far downstream of our area of interest in the city center. Further, this well is a drinking water production well and can therefore not be used as an outflow boundary.  

Few groundwater observation wells are operated by the cantonal office of the environment (AWEL) [\[14\]](#References) (Figure 19).

In [ ]:
du.display_image(
    image_filename='GWmonitoring_locations_AWEL.png', 
    image_folder='1_perceptual_model',
    caption='Figure 19: Monitoring wells in the Limmat valley aquifer. Yearbook sheets for each site can be accessed through the popup window from each site. Source: https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/grundwasserstaende.html, accessed 2025-05-01.'
)

> **🤔 Think about it**  
> What are the major hydrological processes in the Limmat valley aquifer?  
> Where would you set the boundaries of the Limmat valley aquifer?

Now let’s see if we can detect storage changes in the aquifer over time by comparing groundwater levels in the observation wells. For this purpose, we can use groundwater level data from the cantonal monitoring wells.

We have already extracted the groundwater level data from the cantonal monitoring wells published in the link above and can now analyze it to identify any trends or changes in storage over time. If you are interested to learn how to extract the data, please refer to the additional material in [extract_awel_gw_data.ipynb](../ADDITIONNAL_MATERIAL/extract_GW_levels_from_yearbooks/extract_awel_gw_data.ipynb) notebook.

In [ ]:
# Download the data if it doesn't exist
gw_ts_path = download_named_file(
    name='groundwater_timeseries',
    data_type='time_series'
)

# Import the data
gw_ts_raw = pd.read_csv(gw_ts_path)
# Only keep relevant columns
gw_ts = gw_ts_raw[['date', 'value', 'well_id']]
# Make sure date is in a recognized date format
gw_ts.loc[:, 'date'] = pd.to_datetime(gw_ts['date'], errors='coerce')
# Drop duplicates
gw_ts = gw_ts.drop_duplicates()
# Ensure sorted by time
gw_ts = gw_ts[['date', 'well_id', 'value']].sort_values('date')
# The coordinates of IDs B5-3 and B53 mark the same location (albeit in 
# different coordinate systems). We'll use the newer name B5-3 for both.
gw_ts.loc[gw_ts['well_id'] == 'B53', 'well_id'] = 'B5-3'


In [ ]:
# Plot values against date, grouped by well_id
plt.figure(figsize=(12, 6))
for well_id, group in gw_ts.groupby('well_id'):
    plt.plot(group['date'], group['value'], label=well_id)
plt.xlabel('Date')
plt.ylabel('Groundwater Level (m.a.s.l.)')
plt.title('Groundwater Levels Over Time by Well')
plt.legend()
plt.grid()
plt.show()

Let's zoom into the standardized time series to see if the system is in a steady state. 

In [ ]:
# Normalize the values by well_id
gw_ts_std = gw_ts.copy()
gw_ts_std['value'] = gw_ts_std.groupby('well_id')['value'].transform(lambda x: (x - x.mean()) / x.std())

# Monthly mean per well (month start periods). Produces NaN for months with no data.
monthly = ( 
    gw_ts_std 
        .assign( well_id=gw_ts_std['well_id'].astype('string'), date=pd.to_datetime(gw_ts_std['date']) ) 
        .groupby( ['well_id', pd.Grouper(key='date', freq='MS')], as_index=False )['value'] 
        .mean() 
        .rename(columns={'value': 'value_monthly'}) 
)

# Annual mean per well
annual = ( 
    gw_ts_std 
        .assign( well_id=gw_ts_std['well_id'].astype('string'), date=pd.to_datetime(gw_ts_std['date']) ) 
        .groupby( ['well_id', pd.Grouper(key='date', freq='YS')], as_index=False )['value'] 
        .mean() 
        .rename(columns={'value': 'value_annual'}) 
)

# Create the figure
fig = px.line(
    annual,
    x='date',
    y='value_annual',
    color='well_id',
    render_mode='webgl',           # GPU-accelerated for many points
)

# Add range slider + quick selectors for past periods
fig.update_xaxes(
    rangeslider=dict(visible=True),
    rangeselector=dict(
        buttons=list([
            dict(count=5,  label='5Y',  step='year',  stepmode='backward'),
            dict(count=10,  label='10Y',  step='year', stepmode='backward'),
            dict(count=20,  label='20Y',  step='year',  stepmode='backward'),
            dict(step='all', label='All')
        ])
    )
)

fig.update_layout(
    title='Figure X: St. Groundwater Levels Over Time by Well',
    xaxis_title='Date',
    yaxis_title='Standardized Groundwater Level (m.a.s.l.)',
    hovermode='x unified',
    height=520,
    legend_title_text='Well',
    margin=dict(l=40, r=10, t=50, b=40)
)

fig.show()

Looking at the past 20 years of average groundwater table data, no significant trends are visible. We can assume that the groundwater levels have remained relatively stable over this period and that the system is in steady state.

$$ \Delta S = 0 $$

In [ ]:
create_section_completion_marker(5)  

## Summary

Let's update the perceptual model with the information we gathered so far (Figure 22).

In [ ]:
# Calculate the missing fluxes for the water balance
net_recharge = 0.2 * model_area_km2 * 1000000  # m³/year, assuming 200 mm/year net recharge
river_sihl_inflow = sihl_area * q_riv_sihl * 365 * 24 * 3600  # m³/year
river_limmat_inflow = limmat_area * q_riv_limmat * 365 * 24 * 3600  # m³/year
gw_outflow_west = (outflow_lower_bound + outflow_upper_bound) / 2 * 365  # m³/year, average of lower and upper bound

# Create a summary table with the most relevant in- and outflows of the model domain
inflow_summary = {
    "Component": [
        "Net areal recharge to the aquifer (R)",
        "Lateral inflow from the south",
        "Lateral inflow from the north",
        "River Sihl (net inflow)",
        "River Limmat (net inflow)",
    ], 
    "Value (10^6 m³/year)": [
        round(net_recharge / 1e6, 1),
        round(inflow_south / 1e6, 1),
        round(inflow_north / 1e6, 1),
        round(river_sihl_inflow / 1e6, 1),
        round(river_limmat_inflow / 1e6, 1)
    ]
}
outflow_summary = {
    "Component": [
        "Groundwater outflow in the west",
        "Groundwater pumping"
    ], 
    "Value (10^6 m³/year)": [
        round(gw_outflow_west / 1e6, 1),
        round(outflow_pumping / 1e6, 1)
    ]
}

inflow_summary_df = pd.DataFrame(inflow_summary)
outflow_summary_df = pd.DataFrame(outflow_summary)

print("Inflows:")
print(inflow_summary_df)
print("")
print("Outflows:")
print(outflow_summary_df)
print("")
print(f"Sum of inflows: {inflow_summary_df['Value (10^6 m³/year)'].sum():.1f} 10^6 m³/year")
print(f"Sum of outflows: {outflow_summary_df['Value (10^6 m³/year)'].sum():.1f} 10^6 m³/year")
print(f"Difference (inflows - outflows): {(inflow_summary_df['Value (10^6 m³/year)'].sum() - outflow_summary_df['Value (10^6 m³/year)'].sum()):.1f} 10^6 m³/year")

In [ ]:
print("")
print("Inflows (m³/day):")
inflow_summary_df['Value (m³/day)'] = (inflow_summary_df['Value (10^6 m³/year)'] * 1e6 / 365).round().astype(int)
print(inflow_summary_df[['Component', 'Value (m³/day)']])
print("")
print("Outflows (m³/day):")
outflow_summary_df['Value (m³/day)'] = (outflow_summary_df['Value (10^6 m³/year)'] * 1e6 / 365).round().astype(int)
print(outflow_summary_df[['Component', 'Value (m³/day)']])
print("")
print(f"Sum of inflows: {inflow_summary_df['Value (m³/day)'].sum():,} m³/day")
print(f"Sum of outflows: {outflow_summary_df['Value (m³/day)'].sum():,} m³/day")
print(f"Difference (inflows - outflows): {(inflow_summary_df['Value (m³/day)'].sum() - outflow_summary_df['Value (m³/day)'].sum()):,} m³/day")

We currently observe much higher inflows than outflows, but that is acceptable for this first estimate of the fluxes. The purpose of this exercise was to gain a rough understanding of the groundwater budget in the Limmat Valley aquifer. We already know that river–aquifer interactions are complex and cannot be adequately estimated with the data currently available.

In a real-world project, you would need to revisit your river–aquifer interaction model and evaluate whether the chosen parameterization can be adjusted within physically meaningful limits, or whether additional data is required.

In [ ]:
# Display figure with updated fluxes
du.display_image(
    image_filename='perceptual_model_01_final.png', 
    image_folder='1_perceptual_model', 
    caption='Figure 22: Final perceptual model of the Limmat valley aquifer with estimated fluxes. The purple arrows indicate important groundwater fluxes. Qin represent lateral inflows, while Qout represents outflows. Qpump represents groundwater extraction. Qrech and Qleak stand for areal recharge and leakage from surface water bodies respectively. ∆S stands for the storage change in the aquifer.'
)

### Uncertainty of Flux Estimates

Our perceptual model provides a first approximation of groundwater fluxes in the Limmat Valley aquifer. However, all estimated fluxes are subject to considerable uncertainty that should be explicitly acknowledged. Below we summarize the main sources of uncertainty (note that the uncertainty ranges are themselves approximate):

**Net areal recharge (±50%)** : Applying a uniform value of 10% of precipitation across the entire model domain is a major simplification. Recharge in urban areas is highly variable due to impervious surfaces, leaking infrastructure, and artificial drainage. Reported values for urban recharge in the literature range from 5% to 30% of precipitation.

**Lateral inflow from hills (±40%)** : Our simplified catchment delineation and assumed recharge coefficient introduce uncertainty. Actual subsurface flow paths and recharge rates in hillslopes likely vary both spatially and temporally.

**River–aquifer interaction (±70%)** : This component carries the greatest uncertainty due to:
- Simplified representation of river geometry and bed elevation
- Assumptions about leakage coefficients derived from literature
- Spatial variability of riverbed conductance (e.g., clogging layer thickness)
- Temporal dynamics of river stages not represented in the steady-state model
- Uncertainty about connectivity status (connected vs. disconnected conditions)

**Groundwater pumping (±20%)**: Although concession data are available, actual extraction rates are often much lower than the permitted values.

**Groundwater outflow (±50%)** : Our Darcy-based estimate relies on assumed hydraulic conductivity values, which may vary by up to an order of magnitude.

In [ ]:
create_section_completion_marker(6)  

## References
[\[1\]](#1---The-Limmat-Valley-Aquifer) Hug J., and Beilick, A. (1934): Die Grundwasserverhältnisse des Kantons Zürich. In: Beiträge zur Geologie der Schweiz - Geotechnische Serie - Hydrologie. German. Available online here: https://scnat.ch/de/uuid/i/0bd7aa3b-0bd7-5d54-9e2f-597a42dada50-Die_Grundwasserverh%C3%A4ltnisse_des_Kantons_Z%C3%BCrich (accessed 2025-05-01)      
[\[2\]](#1---The-Limmat-Valley-Aquifer) Doppler, T., Hendricks Franssen, H.-J., Kaiser H.-P., Kuhlman U., Stauffer, F. (2007): Field evidence of a dynamic leakage coefficient for modelling river–aquifer interactions. Journal of Hydrology, Volume 347, Issues 1–2, DOI: https://doi.org/10.1016/j.jhydrol.2007.09.017.    
[\[3\]](#1---The-Limmat-Valley-Aquifer) GIS-browser of the canton of Zurich: https://www.gis.zh.ch/ (accessed 2025-05-01)    
[\[4\]](#22---Top-Topography) Federal Office of Topography swisstopo, DHM25 height model, published 2024-01-08, [https://www.swisstopo.admin.ch/en/height-model-dhm25](https://www.swisstopo.admin.ch/en/height-model-dhm25) (accessed 2025-05-01)    
[\[5\]](#23---Lateral-Model-Boundaries) Freeze, R. A., and Cherry, J. A. (1979): Groundwater. Prentice-Hall, Englewood Cliffs, New Jersey, USA. ISBN: 978-0133653122. Open-book available here: [https://gw-project.org/books/groundwater/](https://gw-project.org/books/groundwater/)   
[\[6\]](#3---Climate-Forcing) MeteoSwiss: https://www.meteoswiss.admin.ch/services-and-publications/applications/measurement-values-and-measuring-networks.html#param=messnetz-klima&table=false&station=SMA&compare=y&chart=year (accessed 2025-05-01)  
[\[7\]](#3---Climate-Forcing) Opendata.swiss: Klimanormwerte, publisher: Federal Office of Meterology and Climatology. [https://opendata.swiss/en/dataset/klimanormwerte](https://opendata.swiss/en/dataset/klimanormwerte) (accessed 2025-05-01)    
[\[8\]](#3---Climate-Forcing) Lerner, David N. (2002): Identifying and quantifying urban recharge: a review. Hydrogeology Journal, Volume 10, Issue 1, pp 143–152. DOI: [https://doi.org/10.1007/s10040-001-0177-7](https://doi.org/10.1007/s10040-001-0177-1)    
[\[9\]](#3---Climate-Forcing) Qui, Guo Yu, Yan, Chunhua, Liu, Yuanbo (2023): Urban evapotranspiration and its effects on water budget and energy balance: Review and perspectives. Earth-Science Reviews, Volume 246. DOI: [https://doi.org/10.1016/j.earscirev.2023.104577](https://doi.org/10.1016/j.earscirev.2023.104577)       
[\[10\]](#4---River-aquifer-interaction) Locations of hydrological gauging stations maintained by the Federal Office for the Environment (FOEN): https://map.geo.admin.ch (accessed 2025-05-01)  
[\[11\]](#4---River-aquifer-interaction) Locations of hydrological gauging stations maintained by the cantonal office for waste, water, energy, and air (AWEL): https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/abfluss-wasserstand.html (accessed 2025-05-01)   
[\[12\]](#4---River-aquifer-interaction) Doppler, T., Hendricks Franssen, H.-J., Kaiser H.-P., Kuhlman U., Stauffer, F. (2007): Field evidence of a dynamic leakage coefficient for modelling river–aquifer interactions. Journal of Hydrology, Volume 347, Issues 1–2, DOI: https://doi.org/10.1016/j.jhydrol.2007.09.017.   
[\[13\]](#5---Groundwater-pumping) City of Zurich, Drinking Water Supply: [https://www.stadt-zuerich.ch/de/umwelt-und-energie/wasser.html](https://www.stadt-zuerich.ch/de/umwelt-und-energie/wasser.html) (accessed 2025-08-21)    
[\[14\]](#5---Groundwater-pumping) Statistisches Jahrbuch der Stadt Zürich 2017 (Statistical yearbook of the city of Zurich 2017), Kapitel Wasser und Energie (Chapter water and energy), page 8: Wasserabgabe nach Wasserherkunft; Grundwasser (water deliveries by origin; groundwater).  [https://www.stadt-zuerich.ch/de/politik-und-verwaltung/statistik-und-daten/publikationen-und-dienstleistungen/publikationen/jahrbuch.html](https://www.stadt-zuerich.ch/de/politik-und-verwaltung/statistik-und-daten/publikationen-und-dienstleistungen/publikationen/jahrbuch.html) (accessed 2025-08-21)      
[\[13\]](#6---Monitoring-the-Limmat-Valley-Aquifer) Locations of groundwater monitoring wells maintained by Federal Office for the Environment (FOEN): https://www.zh.ch/de/umwelt-und-natur/wasser/grundwasser/monitoring.html (accessed 2025-05-01)  
[\[14\]](#6---Monitoring-the-Limmat-Valley-Aquifer) Cantonal office of the environment (AWEL): https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/grundwasserstaende.html (accessed 2025-05-01)
